# Hosting Strands Agents with Amazon Bedrock models in Amazon Bedrock AgentCore Runtime

## Overview

In this tutorial we will learn how to host your existing agent, using Amazon Bedrock AgentCore Runtime. We will provide examples using Amazon Bedrock models and non-Bedrock models such as Azure OpenAI and Gemini.


### Tutorial Details


| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                                   |
| Agent type          | Single                                                                           |
| Agentic Framework   | Strands Agents                                                                   |
| Tutorial components | Hosting agent on AgentCore Runtime. Using Strands Agent and Amazon Bedrock Model |
| Tutorial vertical   | Cross-vertical                                                                   |
| Example complexity  | Easy                                                                             |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3                                     |



### Tutorial Key Features

* Hosting Agents on Amazon Bedrock AgentCore Runtime
* Using Amazon Bedrock models
* Using Strands Agents


In [3]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

### Configure AgentCore Runtime deployment

First we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [11]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "resturant_strands_agent"
response = agentcore_runtime.configure(
    entrypoint="restaurant_strands_agent.py",
    execution_role="arn:aws:iam::175918693907:role/_GenAIActivationDay_role",
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

Entrypoint parsed: file=/Users/sbattoo/Src/AIActivationDay/strands_from_scratch/03-resturant-agent-deployto-AgentCore/restaurant_strands_agent.py, bedrock_agentcore_name=restaurant_strands_agent
Configuring BedrockAgentCore agent: resturant_strands_agent


⚠️  ℹ️  No container engine found (Docker/Finch/Podman not installed)
✅ Default deployment uses CodeBuild (no container engine needed)
💡 Run 'agentcore launch' for cloud-based building and deployment
💡 For local builds, install Docker, Finch, or Podman

Generated Dockerfile: /Users/sbattoo/Src/AIActivationDay/strands_from_scratch/03-resturant-agent-deployto-AgentCore/Dockerfile
Generated .dockerignore: /Users/sbattoo/Src/AIActivationDay/strands_from_scratch/03-resturant-agent-deployto-AgentCore/.dockerignore
Keeping 'resturant_strands_agent' as default agent
Bedrock AgentCore configured: /Users/sbattoo/Src/AIActivationDay/strands_from_scratch/03-resturant-agent-deployto-AgentCore/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/Users/sbattoo/Src/AIActivationDay/strands_from_scratch/03-resturant-agent-deployto-AgentCore/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/Users/sbattoo/Src/AIActivationDay/strands_from_scratch/03-resturant-agent-deployto-AgentCore/Dockerfile'), dockerignore_path=PosixPath('/Users/sbattoo/Src/AIActivationDay/strands_from_scratch/03-resturant-agent-deployto-AgentCore/.dockerignore'), runtime='None', region='us-west-2', account_id='175918693907', execution_role='arn:aws:iam::175918693907:role/_GenAIActivationDay_role', ecr_repository=None, auto_create_ecr=True)

### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [12]:
launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'resturant_strands_agent' to account 175918693907 (us-west-2)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: resturant_strands_agent
✅ ECR repository available: 175918693907.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-resturant_strands_agent
Using execution role from config: arn:aws:iam::175918693907:role/_GenAIActivationDay_role
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: resturant_strands_agent
Role name: AmazonBedrockAg

✅ Reusing existing ECR repository: 175918693907.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-resturant_strands_agent


Reusing existing CodeBuild execution role: arn:aws:iam::175918693907:role/AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-b382033528
Using .dockerignore with 44 patterns
Uploaded source to S3: resturant_strands_agent/source.zip
Updated CodeBuild project: bedrock-agentcore-resturant_strands_agent-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.1s
🔄 PROVISIONING started (total: 1s)
✅ PROVISIONING completed in 10.6s
🔄 DOWNLOAD_SOURCE started (total: 12s)
✅ DOWNLOAD_SOURCE completed in 2.1s
🔄 BUILD started (total: 14s)
✅ BUILD completed in 23.5s
🔄 POST_BUILD started (total: 37s)
✅ POST_BUILD completed in 17.0s
🔄 FINALIZING started (total: 54s)
✅ FINALIZING completed in 1.1s
🔄 COMPLETED started (total: 55s)
✅ COMPLETED completed in 1.1s
🎉 CodeBuild completed successfully in 0m 56s
CodeBuild completed successfully
✅ CodeBuild project configuration saved
Deploying to Bedrock AgentCore...
⚠️ 

### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [13]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: resturant_strands_agent


'READY'

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [14]:
invoke_response = agentcore_runtime.invoke({"prompt": "Hi, where can I eat in San Francisco?"})
invoke_response

{'ResponseMetadata': {'RequestId': '935dc2d2-17dc-42b0-b621-574f6000a816',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 18 Sep 2025 23:32:44 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': '935dc2d2-17dc-42b0-b621-574f6000a816',
   'baggage': 'Self=1-68cc967f-745a93d539d061dc305e2a34,session.id=ef8cc89a-28ed-4c6d-adcb-1176ffd59756',
   'x-amzn-bedrock-agentcore-runtime-session-id': 'ef8cc89a-28ed-4c6d-adcb-1176ffd59756',
   'x-amzn-trace-id': 'Root=1-68cc967f-0698582d0ea366d964ecb806;Self=1-68cc967f-745a93d539d061dc305e2a34'},
  'RetryAttempts': 0},
 'runtimeSessionId': 'ef8cc89a-28ed-4c6d-adcb-1176ffd59756',
 'traceId': 'Root=1-68cc967f-0698582d0ea366d964ecb806;Self=1-68cc967f-745a93d539d061dc305e2a34',
 'baggage': 'Self=1-68cc967f-745a93d539d061dc305e2a34,session.id=ef8cc89a-28ed-4c6d-adcb-1176ffd59756',
 'contentType': 'application/json',
 'statusCode': 200,
 'response': ["<answer>\nHel

In [15]:
invoke_response = agentcore_runtime.invoke({"prompt": "Make a reservation for tonight at Rice & Spice"})
invoke_response

{'ResponseMetadata': {'RequestId': '8a72ab1f-d21c-4f96-8778-3aa66747f661',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 18 Sep 2025 23:33:53 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': '8a72ab1f-d21c-4f96-8778-3aa66747f661',
   'baggage': 'Self=1-68cc96de-64401e5706c075051e66486b,session.id=ef8cc89a-28ed-4c6d-adcb-1176ffd59756',
   'x-amzn-bedrock-agentcore-runtime-session-id': 'ef8cc89a-28ed-4c6d-adcb-1176ffd59756',
   'x-amzn-trace-id': 'Root=1-68cc96de-637a2772214d470e674ab163;Self=1-68cc96de-64401e5706c075051e66486b'},
  'RetryAttempts': 0},
 'runtimeSessionId': 'ef8cc89a-28ed-4c6d-adcb-1176ffd59756',
 'traceId': 'Root=1-68cc96de-637a2772214d470e674ab163;Self=1-68cc96de-64401e5706c075051e66486b',
 'baggage': 'Self=1-68cc96de-64401e5706c075051e66486b,session.id=ef8cc89a-28ed-4c6d-adcb-1176ffd59756',
 'contentType': 'application/json',
 'statusCode': 200,
 'response': ["I'd be happy 

In [17]:
invoke_response = agentcore_runtime.invoke({"prompt": "At 10pm today, for 5 people in the name of Sameer"})
invoke_response

{'ResponseMetadata': {'RequestId': 'c5979889-7556-4c80-87be-2616d0db4029',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 18 Sep 2025 23:34:44 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c5979889-7556-4c80-87be-2616d0db4029',
   'baggage': 'Self=1-68cc9709-7cb4a9571c2aa8dc180994a6,session.id=ef8cc89a-28ed-4c6d-adcb-1176ffd59756',
   'x-amzn-bedrock-agentcore-runtime-session-id': 'ef8cc89a-28ed-4c6d-adcb-1176ffd59756',
   'x-amzn-trace-id': 'Root=1-68cc9709-01b503e50a83a05c227e5311;Self=1-68cc9709-7cb4a9571c2aa8dc180994a6'},
  'RetryAttempts': 0},
 'runtimeSessionId': 'ef8cc89a-28ed-4c6d-adcb-1176ffd59756',
 'traceId': 'Root=1-68cc9709-01b503e50a83a05c227e5311;Self=1-68cc9709-7cb4a9571c2aa8dc180994a6',
 'baggage': 'Self=1-68cc9709-7cb4a9571c2aa8dc180994a6,session.id=ef8cc89a-28ed-4c6d-adcb-1176ffd59756',
 'contentType': 'application/json',
 'statusCode': 200,
 'response': ["<answer>\nTha

### Invoking AgentCore Runtime with boto3

Now that your AgentCore Runtime was created you can invoke it with any AWS SDK. For instance, you can use the boto3 `invoke_agent_runtime` method for it.

In [18]:
import boto3
from IPython.display import Markdown, display
import json
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "Hi, Suggest some places where I can get Vegan?"})
)

if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))





<answer>
Restaurant Helper here! Based on my search, I'd highly recommend Botanic Table, which is a fully plant-based/vegan restaurant located at 1845 Garden Avenue in Portland, OR. 

Their menu includes delicious vegan options such as:

Small Plates:
- Heirloom Tomato Carpaccio with black garlic and basil oil ($12)
- Charred Brussels Sprouts with maple glaze and smoked almond ($10)
- Watermelon "Poke" with avocado, cucumber, and ponzu ($13)

Large Plates:
- King Oyster Mushroom "Scallops" with cauliflower purée ($19)
- Jackfruit Barbacoa Tacos with cashew crema ($16)
- Ancient Grain Bowl with seasonal vegetables and miso dressing ($17)
- Charred Eggplant Steak with romesco sauce ($18)

They also offer vegan desserts including Coconut Milk Panna Cotta, Dark Chocolate Avocado Mousse, and Cashew Cheesecake.

Would you like me to help you make a reservation at Botanic Table or provide information about any other restaurants?
</answer>

## Congrats!

Congrats, you've deployed and invoked you agent hosted on AgentCore Runtime. As optional step, you can delete the prerequisite infrastructure created

In [ ]:
# !sh cleanup.sh